In [1]:
#!/usr/bin/env python
import os, datetime
import pandas as pd
import numpy as np
import requests

pd.options.display.max_columns = None

# Show multiple outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Import US data at the state and county level
+ Social distancing measures (University of Washington, NGA & KFF): https://github.com/COVID19StatePolicy/SocialDistancing
+ case_death_df of confirmed cases and deaths (John Hopkins University): https://github.com/CSSEGISandData/COVID-19 
+ Google community mobility data: https://www.google.com/covid19/mobility/

In [2]:
#Import latest data using the links
meas_url = 'https://raw.githubusercontent.com/COVID19StatePolicy/SocialDistancing/master/data/USstatesCov19distancingpolicy.csv'
case_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv'
deaths_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv'
mob_url = 'https://www.gstatic.com/covid19/mobility/Global_Mobility_Report.csv'

measures = pd.read_csv(meas_url,header = 0,index_col=0,encoding = "ISO-8859-1", engine='python',
                   parse_dates=['DateIssued','DateEnacted','DateExpiry','DateEased','DateEnded'])#csv used to have 582 empty rows at the end, check
cases = pd.read_csv(case_url,header = 0,engine='python')
deaths = pd.read_csv(deaths_url,header = 0,engine='python')
mobility = pd.read_csv(mob_url,parse_dates=['date'])


/Users/bpeshlov/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Check the dataframes

In [9]:
#Keep only US mobility rows
mobility_us = mobility[mobility.country_region_code=='US']

In [11]:
### check dataframes for data types and empty rows
measures.info()
measures.isnull().sum(axis=0)
print('Cases df Shape:\n', cases.shape)
#cases.isnull().sum(axis=0)
print('Deaths df Shape:\n', deaths.shape)
#deaths.isnull().sum(axis=0)
mobility_us.info()
mobility_us.isnull().sum(axis=0)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 581 entries, 523 to 573
Data columns (total 15 columns):
StateFIPS            581 non-null int64
StatePostal          581 non-null object
StateName            581 non-null object
StatePolicy          581 non-null object
Mandate              581 non-null int64
DateIssued           581 non-null datetime64[ns]
DateEnacted          581 non-null datetime64[ns]
DateExpiry           328 non-null datetime64[ns]
DateEased            125 non-null datetime64[ns]
DateEnded            29 non-null datetime64[ns]
PolicyCodingNotes    536 non-null object
PolicySource         581 non-null object
StateWide            581 non-null int64
LastUpdated          581 non-null int64
LastUpdatedNotes     425 non-null object
dtypes: datetime64[ns](5), int64(4), object(6)
memory usage: 72.6+ KB


StateFIPS              0
StatePostal            0
StateName              0
StatePolicy            0
Mandate                0
DateIssued             0
DateEnacted            0
DateExpiry           253
DateEased            456
DateEnded            552
PolicyCodingNotes     45
PolicySource           0
StateWide              0
LastUpdated            0
LastUpdatedNotes     156
dtype: int64

Cases df Shape:
 (3261, 129)


UID                0
iso2               0
iso3               0
code3              0
FIPS              10
Admin2             7
Province_State     0
Country_Region     0
Lat                0
Long_              0
Combined_Key       0
1/22/20            0
1/23/20            0
1/24/20            0
1/25/20            0
1/26/20            0
1/27/20            0
1/28/20            0
1/29/20            0
1/30/20            0
1/31/20            0
2/1/20             0
2/2/20             0
2/3/20             0
2/4/20             0
2/5/20             0
2/6/20             0
2/7/20             0
2/8/20             0
2/9/20             0
                  ..
4/19/20            0
4/20/20            0
4/21/20            0
4/22/20            0
4/23/20            0
4/24/20            0
4/25/20            0
4/26/20            0
4/27/20            0
4/28/20            0
4/29/20            0
4/30/20            0
5/1/20             0
5/2/20             0
5/3/20             0
5/4/20             0
5/5/20       

Deaths df Shape:
 (3261, 130)


UID                0
iso2               0
iso3               0
code3              0
FIPS              10
Admin2             7
Province_State     0
Country_Region     0
Lat                0
Long_              0
Combined_Key       0
Population         0
1/22/20            0
1/23/20            0
1/24/20            0
1/25/20            0
1/26/20            0
1/27/20            0
1/28/20            0
1/29/20            0
1/30/20            0
1/31/20            0
2/1/20             0
2/2/20             0
2/3/20             0
2/4/20             0
2/5/20             0
2/6/20             0
2/7/20             0
2/8/20             0
                  ..
4/19/20            0
4/20/20            0
4/21/20            0
4/22/20            0
4/23/20            0
4/24/20            0
4/25/20            0
4/26/20            0
4/27/20            0
4/28/20            0
4/29/20            0
4/30/20            0
5/1/20             0
5/2/20             0
5/3/20             0
5/4/20             0
5/5/20       

<class 'pandas.core.frame.DataFrame'>
Int64Index: 239694 entries, 108182 to 347875
Data columns (total 11 columns):
country_region_code                                   239694 non-null object
country_region                                        239694 non-null object
sub_region_1                                          239605 non-null object
sub_region_2                                          235066 non-null object
date                                                  239694 non-null datetime64[ns]
retail_and_recreation_percent_change_from_baseline    177876 non-null float64
grocery_and_pharmacy_percent_change_from_baseline     170460 non-null float64
parks_percent_change_from_baseline                    62506 non-null float64
transit_stations_percent_change_from_baseline         93474 non-null float64
workplaces_percent_change_from_baseline               229640 non-null float64
residential_percent_change_from_baseline              109248 non-null float64
dtypes: datetime64[ns](1)

country_region_code                                        0
country_region                                             0
sub_region_1                                              89
sub_region_2                                            4628
date                                                       0
retail_and_recreation_percent_change_from_baseline     61818
grocery_and_pharmacy_percent_change_from_baseline      69234
parks_percent_change_from_baseline                    177188
transit_stations_percent_change_from_baseline         146220
workplaces_percent_change_from_baseline                10054
residential_percent_change_from_baseline              130446
dtype: int64

### Prepare US states mobility data (missing county names)

In [49]:
#make a new df with no NaNs in states column where sub_region_1=states, sub_region_2 = counties
mobility_counties = mobility_us[mobility_us['sub_region_1'].notna()]#all rows where state is not NAN
mobility_states = mobility_counties[mobility_counties['sub_region_2'].isna()]#keep only rows with states and no county value (state only rows)

In [50]:
mobility_states.drop(['country_region_code','country_region','sub_region_2'], axis=1,inplace=True)
mobility_states.info()
#mobility_counties.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4539 entries, 108271 to 346165
Data columns (total 8 columns):
sub_region_1                                          4539 non-null object
date                                                  4539 non-null datetime64[ns]
retail_and_recreation_percent_change_from_baseline    4539 non-null float64
grocery_and_pharmacy_percent_change_from_baseline     4539 non-null float64
parks_percent_change_from_baseline                    4535 non-null float64
transit_stations_percent_change_from_baseline         4539 non-null float64
workplaces_percent_change_from_baseline               4539 non-null float64
residential_percent_change_from_baseline              4539 non-null float64
dtypes: datetime64[ns](1), float64(6), object(1)
memory usage: 319.1+ KB


In [52]:
#check for any missing or misspelled states before joining cases, measures and mobility
print(set(mobility_states.loc[:,'sub_region_1']) - set(cases.loc[:,'Province_State']))
print(set(measures.loc[:,'StateName'])-set(cases.loc[:,'Province_State']))

## Check for 'empty strings'
## If this generate non-empty array, then dataset contains empty strings.
#print(mobility_states[mobility_states.applymap(lambda x: x == '').any(axis=1)])
#print(cases[cases.applymap(lambda x: x == '').any(axis=1)])
#print(deaths[deaths.applymap(lambda x: x == '').any(axis=1)])
#print(measures[measures.applymap(lambda x: x == '').any(axis=1)])
#print("If the dataframe above is empty, then it doesn't have any empty strings")

set()


In [53]:
#remove suffix from mobility columns
mobility_states.columns = [col.replace('_percent_change_from_baseline', '') for col in mobility_states.columns]

### Preprocess measures df before merging

In [22]:
#Select columns from measures df to merge with cases and deaths dfs
measures=measures[~measures.StateFIPS.isnull()]#drop rows with empty StateFIPS values
#select measures columns of interest
meas_sel = measures[['StateName','StatePolicy','Mandate','DateIssued','DateEnacted','StateWide',
                     'DateExpiry','DateEased','DateEnded']]

,StateName,StatePolicy,Mandate,DateIssued,DateEnacted,StateWide,DateExpiry,DateEased,DateEnded
location_id,,,,,,,,,
523,Alabama,EmergDec,1,2020-03-13,2020-03-13,1,NaT,NaT,NaT
523,Alabama,SchoolClose,1,2020-03-14,2020-03-18,1,NaT,NaT,NaT
523,Alabama,GathRestrict25,1,2020-03-19,2020-03-19,1,NaT,NaT,NaT
523,Alabama,GathRestrictAny,1,2020-03-19,2020-03-19,1,2020-05-22,NaT,NaT
523,Alabama,OtherBusinessClose,1,2020-03-19,2020-03-19,1,2020-05-22,2020-04-30,NaT


In [23]:
#Check measures df again for missing values
meas_sel.isnull().sum(axis=0)# total NaNs per column 
len(meas_sel.index)#total rows
#most missing values are in DateEnacted
#Iowa SchoolClose has DateIssued but DateEnacted is NaT

StateName        0
StatePolicy      0
Mandate          0
DateIssued       0
DateEnacted      0
StateWide        0
DateExpiry     253
DateEased      456
DateEnded      552
dtype: int64

581

,StateName,StatePolicy,Mandate,DateIssued,DateEnacted,StateWide,DateExpiry,DateEased,DateEnded
location_id,,,,,,,,,


### Preprocess and merge cases and deaths dfs

In [25]:
#drop columns initally not needed for modeling
cases.drop(['iso2','iso3','Country_Region','Combined_Key','UID','code3','FIPS','Lat','Long_'], axis=1,inplace=True)
deaths.drop(['iso2','iso3','Country_Region','Combined_Key','UID','code3','FIPS','Lat','Long_'], axis=1,inplace=True)

In [26]:
#Reshape cases and deaths df from wide to tall format
c_melt = cases.melt(id_vars =['Province_State', 'Admin2'],var_name='date',value_name='cases')
d_melt = deaths.melt(id_vars =['Province_State', 'Admin2','Population'],var_name='date',value_name='deaths')

In [140]:
#merge cases and deaths df on state, county and date columns
case_death_df = pd.merge(c_melt,d_melt,how='left',on=['Province_State','Admin2','date'])
#convert date colum from str to date
case_death_df['date'] = pd.to_datetime(case_death_df['date'], format = "%m/%d/%y")

### Merge cases, deaths, measures and mobility

In [141]:
#Add columns with 0s for each measure to the main df
dfzeros=pd.DataFrame(np.zeros((len(case_death_df),len(meas_sel.StatePolicy.unique()))),columns = list(meas_sel.StatePolicy.unique())).astype(int)
tseries = pd.concat([case_death_df, dfzeros], axis=1)
tseries.columns = tseries.columns.str.replace('Province_State','state').str.replace('Admin2','county')

### Add measures categorical columns

In [143]:
#Add measures to cases + deaths df

#Loop over states and measures. Plug  1s in the dates and state row each measure was enacted
for state in meas_sel.StateName.unique():
    for i,meas in enumerate(meas_sel.StatePolicy.unique()):
        #select rows by state and  measure
        mask1 = (meas_sel.StateName== state) & (meas_sel.StatePolicy==meas)
        
        if not meas_sel[mask1].empty:
            #date policy enacted
            start = meas_sel.loc[mask1,"DateEnacted"].values[0]
            #date policy expired
            end =  meas_sel.loc[mask1,"DateEnded"].values[0]
        else:
            print(state+ " is missing " + meas)
            continue
            
        if pd.notnull(start) & pd.notnull(end):
            mask2 = (tseries.state==state) & (tseries.date >= start) & (tseries.date <= end)
        elif pd.notnull(start):
            mask2 = (tseries.state==state) & (tseries.date >= start)
        else:
            continue
        #set measure values to 1 after date was enacted by state
        tseries.loc[mask2, meas] = 1


Alabama is missing Quarantine
Alabama is missing NEBusinessClose
Alabama is missing TravelRestrictIntra
Alabama is missing GathRestrict50
Alabama is missing BusinessHealthSafety
Alabama is missing GathRestrict250
Alabama is missing GathRecomAny
Alabama is missing GathRestrict1000
Alabama is missing TravelRestrictExit
Alabama is missing TravelRestrictEntry
Alabama is missing GathRestrict100
Alabama is missing GathRestrict5
Alabama is missing GathRestrict500
Alaska is missing BusinessHealthSafety
Alaska is missing GathRestrict250
Alaska is missing GathRecomAny
Alaska is missing GathRestrict1000
Alaska is missing TravelRestrictExit
Alaska is missing TravelRestrictEntry
Alaska is missing GathRestrict100
Alaska is missing GathRestrict5
Alaska is missing GathRestrict500
Arizona is missing GathRestrict25
Arizona is missing GathRestrict10
Arizona is missing CaseIsolation
Arizona is missing PublicMask
Arizona is missing NEBusinessClose
Arizona is missing TravelRestrictIntra
Arizona is missing G

Kentucky is missing GathRestrict10
Kentucky is missing CaseIsolation
Kentucky is missing TravelRestrictIntra
Kentucky is missing GathRestrict50
Kentucky is missing BusinessHealthSafety
Kentucky is missing GathRestrict250
Kentucky is missing GathRestrict1000
Kentucky is missing GathRestrict100
Kentucky is missing GathRestrict5
Kentucky is missing GathRestrict500
Louisiana is missing GathRestrict25
Louisiana is missing CaseIsolation
Louisiana is missing Quarantine
Louisiana is missing TravelRestrictIntra
Louisiana is missing BusinessHealthSafety
Louisiana is missing GathRestrict1000
Louisiana is missing TravelRestrictExit
Louisiana is missing TravelRestrictEntry
Louisiana is missing GathRestrict100
Louisiana is missing GathRestrict5
Louisiana is missing GathRestrict500
Maine is missing GathRestrict25
Maine is missing CaseIsolation
Maine is missing TravelRestrictIntra
Maine is missing GathRestrict50
Maine is missing BusinessHealthSafety
Maine is missing GathRestrict250
Maine is missing Ga

Ohio is missing GathRestrict25
Ohio is missing CaseIsolation
Ohio is missing TravelRestrictIntra
Ohio is missing BusinessHealthSafety
Ohio is missing GathRestrict250
Ohio is missing GathRestrict1000
Ohio is missing TravelRestrictExit
Ohio is missing GathRestrict5
Ohio is missing GathRestrict500
Oklahoma is missing GathRestrict25
Oklahoma is missing CaseIsolation
Oklahoma is missing PublicMask
Oklahoma is missing TravelRestrictIntra
Oklahoma is missing GathRestrict50
Oklahoma is missing BusinessHealthSafety
Oklahoma is missing GathRestrict250
Oklahoma is missing GathRestrict1000
Oklahoma is missing TravelRestrictExit
Oklahoma is missing TravelRestrictEntry
Oklahoma is missing GathRestrict100
Oklahoma is missing GathRestrict5
Oklahoma is missing GathRestrict500
Oregon is missing CaseIsolation
Oregon is missing PublicMask
Oregon is missing Quarantine
Oregon is missing NEBusinessClose
Oregon is missing TravelRestrictIntra
Oregon is missing GathRestrict50
Oregon is missing BusinessHealthSaf

In [144]:
# Rename columns and merge tseries and mobility on states and date columns (counties will be NaNs)
df = pd.merge(tseries, mobility_states,  how='left', left_on=['state','date'], right_on = ['sub_region_1','date'])

In [145]:
#Drop duplicate columns
df.drop(['sub_region_1'],axis=1,inplace=True)

In [146]:
#Export final df to csv
df.to_csv('05_19_states_cases_measures_mobility.csv')

In [ ]:
#Save original data
cases.to_csv('jhu_cases_05_19_20_update.csv')
deaths.to_csv('jhu_deaths_05_19_20_update.csv')
measures.to_csv('measures_05_19_20_update.csv')
mobility_us.to_csv('Google_community_mobility_US_05_19_20_update.csv')

In [155]:
#tt = df.loc[df["county"]== "Wyoming", "state"]
tt = tseries.loc[(df["state"]== "Wyoming") & (tseries["GathRestrict10"]==1)]
tt.date.unique()

array(['2020-03-20T00:00:00.000000000', '2020-03-21T00:00:00.000000000',
       '2020-03-22T00:00:00.000000000', '2020-03-23T00:00:00.000000000',
       '2020-03-24T00:00:00.000000000', '2020-03-25T00:00:00.000000000',
       '2020-03-26T00:00:00.000000000', '2020-03-27T00:00:00.000000000',
       '2020-03-28T00:00:00.000000000', '2020-03-29T00:00:00.000000000',
       '2020-03-30T00:00:00.000000000', '2020-03-31T00:00:00.000000000',
       '2020-04-01T00:00:00.000000000', '2020-04-02T00:00:00.000000000',
       '2020-04-03T00:00:00.000000000', '2020-04-04T00:00:00.000000000',
       '2020-04-05T00:00:00.000000000', '2020-04-06T00:00:00.000000000',
       '2020-04-07T00:00:00.000000000', '2020-04-08T00:00:00.000000000',
       '2020-04-09T00:00:00.000000000', '2020-04-10T00:00:00.000000000',
       '2020-04-11T00:00:00.000000000', '2020-04-12T00:00:00.000000000',
       '2020-04-13T00:00:00.000000000', '2020-04-14T00:00:00.000000000',
       '2020-04-15T00:00:00.000000000', '2020-04-16

In [154]:
meas_sel.loc[meas_sel.StateName == "Wyoming"]

,StateName,StatePolicy,Mandate,DateIssued,DateEnacted,StateWide,DateExpiry,DateEased,DateEnded
location_id,,,,,,,,,
573,Wyoming,EmergDec,1,2020-03-13,2020-03-13,1,NaT,NaT,NaT
573,Wyoming,OtherBusinessClose,1,2020-03-19,2020-03-19,1,2020-06-01,2020-05-01,NaT
573,Wyoming,RestaurantRestrict,1,2020-03-19,2020-03-19,1,2020-06-01,2020-05-15,NaT
573,Wyoming,SchoolClose,1,2020-03-19,2020-03-19,1,2020-06-01,NaT,NaT
573,Wyoming,GathRestrict10,1,2020-03-20,2020-03-20,1,2020-05-16,NaT,2020-05-15
573,Wyoming,GathRestrictAny,1,2020-03-20,2020-03-20,1,2020-06-01,2020-05-15,NaT
573,Wyoming,OtherBusinessClose,1,2020-03-24,2020-03-25,1,2020-06-01,2020-05-01,NaT
573,Wyoming,Quarantine,1,2020-04-03,2020-04-03,1,2020-05-09,NaT,NaT
573,Wyoming,PublicMask,0,2020-04-06,2020-04-06,1,NaT,NaT,NaT
